# 1. Extraction 

In [ ]:
import requests
import pandas as pd
import config


## a. Retrieve the data comparing the level of technology use by sector in US
1. Query the US Census API with the call:
   https://api.census.gov/data/2018/abstcb?get=NAME,TECHUSE,TECHUSE_LABEL,NAICS2017,NAICS2017_LABEL,FIRMPDEMP&for=us&key=YOUR_KEY
2. Load the JSON response into a pandas dataframe TechUse_Sector_us

In [ ]:
baseURL = 'https://api.census.gov/data/2018/abstcb'
dataToGet = '?get=NAME,TECHUSE,TECHUSE_LABEL,NAICS2017,NAICS2017_LABEL,FIRMPDEMP'
regionToGet = '&for=us'
surveyToGet = ''
queryURL = baseURL+dataToGet+regionToGet+surveyToGet+'&key='+config.personalKey 
response = requests.get(queryURL)
print(queryURL)
print(response.text)

In [ ]:
TechUseHigh_Sector_us = pd.read_json(response.text)
TechUseHigh_Sector_us.columns = TechUseHigh_Sector_us.iloc[0]
TechUseHigh_Sector_us.drop(index=0, inplace=True)
print(TechUseHigh_Sector_us.shape)
print(TechUseHigh_Sector_us.head(10))
print(TechUseHigh_Sector_us.columns)
print(TechUseHigh_Sector_us.NAICS2017_LABEL.unique())

## b. Retrieve the data comparing the level of technology use by business size in US
1. Query the US Census API with the call:
   https://api.census.gov/data/2018/abstcb?get=NAME,TECHUSE,TECHUSE_LABEL,NSFSZFI,NSFSZFI_LABEL,FIRMPDEMP&for=us&key=YOUR_KEY
2. Load the JSON response into a pandas dataframe TechUse_Size_us

In [ ]:
dataToGet = '?get=NAME,TECHUSE,TECHUSE_LABEL,NSFSZFI,NSFSZFI_LABEL,FIRMPDEMP'
queryURL = baseURL+dataToGet+regionToGet+surveyToGet+'&key='+config.personalKey
response = requests.get(queryURL)
print(queryURL)
print(response.text)

In [ ]:
TechUseHigh_Size_us = pd.read_json(response.text)
TechUseHigh_Size_us.columns = TechUseHigh_Size_us.iloc[0]
TechUseHigh_Size_us.drop(index=0, inplace=True)
print(TechUseHigh_Size_us.shape)
print(TechUseHigh_Size_us.head(10))
print(TechUseHigh_Size_us.columns)
print(TechUseHigh_Size_us.NSFSZFI_LABEL.unique())

## c. Retrieve the data comparing the level of technology use by state
1. Query the US Census API with the call:
   https://api.census.gov/data/2018/abstcb?get=NAME,TECHUSE,TECHUSE_LABEL,FIRMPDEMP&for=state&key=YOUR_KEY
2. Load the JSON response into a pandas dataframe TechUse_state

In [ ]:
dataToGet = '?get=NAME,TECHUSE,TECHUSE_LABEL,FIRMPDEMP'
regionToGet = '&for=state'
queryURL = baseURL+dataToGet+regionToGet+surveyToGet+'&key='+config.personalKey
response = requests.get(queryURL)
print(queryURL)
print(response.text)

In [ ]:
TechUseHigh_state = pd.read_json(response.text)
TechUseHigh_state.columns = TechUseHigh_state.iloc[0]
TechUseHigh_state.drop(index=0, inplace=True)
print(TechUseHigh_state.shape)
print(TechUseHigh_state.head(10))
print(TechUseHigh_state.columns)
print(TechUseHigh_state.NAME.unique())

## d. Retrieve the dataset with the US Census regions and divisions for the states
1. Read the raw csv file from the Github repository at https://github.com/cphalpert/census-regions/blob/master/us%20census%20bureau%20regions%20and%20divisions.csv

In [ ]:
file_path = 'https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv'
Census_regions = pd.read_csv(file_path)
print(Census_regions)

# 2. Transformation

# a. Remove the unneeded rows from the dataframes
1. Remove all rows except TECHUSE==T3E03B05, TECHUSE==T3E03B06, and TECHUSE==T3E03BA9. These are the high use, Don't know, total responses for Artificial Information technology use.
2. Reset indices to sequential

In [ ]:
remainingTECHUSE = ['T1E03B05', 'T1E03B06', 'T1E03BA9']
TechUseHigh_Sector_us = TechUseHigh_Sector_us[TechUseHigh_Sector_us['TECHUSE'].isin(remainingTECHUSE) == True]
print(TechUseHigh_Sector_us)
TechUseHigh_Size_us = TechUseHigh_Size_us[TechUseHigh_Size_us['TECHUSE'].isin(remainingTECHUSE) == True]
print(TechUseHigh_Size_us)
TechUseHigh_state = TechUseHigh_state[TechUseHigh_state['TECHUSE'].isin(remainingTECHUSE) == True]
print(TechUseHigh_state)

In [ ]:
TechUseHigh_Sector_us.reset_index(drop=True, inplace=True)
TechUseHigh_Size_us.reset_index(drop=True, inplace=True)
TechUseHigh_state.reset_index(drop=True, inplace=True)
print(TechUseHigh_Sector_us)
print(TechUseHigh_Size_us)
print(TechUseHigh_state)

# b. Remove the unneeded columns from the dataframes
1. Remove the NAME and 'us' column from the us-based dataframes.
2. Remove the 'state' column from the state-based dataframe.

In [ ]:
TechUseHigh_Sector_us.drop(axis=1, columns=['NAME','us'], inplace=True)
TechUseHigh_Size_us.drop(axis=1, columns=['NAME', 'us'], inplace=True)
TechUseHigh_state.drop(axis=1, columns=['state'], inplace=True)
print(TechUseHigh_Sector_us)
print(TechUseHigh_Size_us)
print(TechUseHigh_state)

## c. Merge state regions with the state-based dataset
1. Change column names in the datasets for the state abbreviations to match. 
2. Do an inner join to add the region and division labels to the state-based dataframe.

In [ ]:
TechUseHigh_state.rename(columns={'NAME': 'State'}, inplace=True)
Census_regions.rename(columns={'State Code': 'StateCode'}, inplace=True)
TechUseHigh_state = TechUseHigh_state.merge(Census_regions, how='inner', on='State')
print(TechUseHigh_state)


# 3. Load

# a. Save files for later use
Save each dataframe as a csv file
TechUseHigh_Sector_us.csv, TechUseHigh_Size_us.csv, and TechUseHigh_state.csv

In [ ]:
TechUseHigh_Sector_us.to_csv('Data/TechUseHigh_Sector_us.csv', index=False)
TechUseHigh_Size_us.to_csv('Data/TechUseHigh_Size_us.csv', index=False)
TechUseHigh_state.to_csv('Data/TechUseHigh_state.csv', index=False)